In [1]:
import requests
import pandas as pd
import time
import os

In [2]:
#get full list of all publicly available app ids on the steam store
url = "http://api.steampowered.com/ISteamApps/GetAppList/v0002/?format=json"

res = requests.get(url)
if res.status_code == 200:
    steam_games = pd.DataFrame(res.json()['applist']['apps']).sort_values(by='appid').reset_index().drop('index', axis=1)
print(steam_games.shape)
steam_games.head()

(132189, 2)


,appid,name
0,5,Dedicated Server
1,7,Steam Client
2,8,winui2
3,10,Counter-Strike
4,20,Team Fortress Classic


In [3]:
steam_games.to_csv('../data/app_ids.csv')

In [4]:
# steam store data collection
def get_store_app_data(appid):
    url = f'http://store.steampowered.com/api/appdetails?appids={appid}'
    res = requests.get(url)
    if res.status_code == 200:
        try:
            return res.json()[f'{appid}']
        except:
            return {'appid':appid, 'error': 'json error', 'api': 'steam store'}
    else:
        print(f'Error code {res.status_code} accessing steam store data for app id {appid}',end='\r')
        return {'appid':appid, 'error': res.status_code, 'api': 'steam store'}

In [5]:
# additional steam spy data
def get_spy_app_data(appid):
    url = f'https://steamspy.com/api.php?request=appdetails&appid={appid}'
    res = requests.get(url)
    if res.status_code == 200:
        try:
            return res.json()
        except:
            return {'appid':appid, 'error': 'json error', 'api': 'steamspy'}
    else:
        print(f'Error code {res.status_code} for app id {appid} via steam spy')
        return {'appid':appid, 'error': res.status_code, 'api': 'steamspy'}

In [6]:
def generate_data_batch(app_list):
    apps = []
    errors = []
    for app_id in app_list:
        store_data = get_store_app_data(app_id)
        try: 
            valid = (store_data['success'] == True) 
        except:
            errors.append(store_data)
            continue
        
        print('                                                                                    ', end='\r')
        print(f'Retrieving data for app id {app_id}... Total apps collected this batch: {len(apps)}',end='\r')
        
        if valid:
            if store_data['data']['type'] == 'game':
                spy_data = get_spy_app_data(app_id)
                try:
                    check = spy_data['name']
                    apps.append(spy_data)
                except:
                    errors.append(spy_data)
        time.sleep(1.5)
    return apps, errors

In [7]:
def pull_data(app_list, start=0, batch_size=600):
    try:
        os.mkdir('../data/batches/')
    except:
        pass
    try:
        os.mkdir('../data/errors/')
    except:
        pass
    
    for i in range(len(app_list)//batch_size):
        batch, fails = generate_data_batch(app_list[start:(start+batch_size)])
        pd.DataFrame(batch).to_csv(f'../data/batches/batch{i}.csv')
        if len(fails) > 0:
            pd.DataFrame(fails).to_csv(f'../data/errors/errors_batch{i}.csv')
        start += batch_size
        
    batch, fails =  generate_data_batch(app_list[start:])
    pd.DataFrame(batch).to_csv(f'../data/batches/batch_final.csv')
    
    if len(fails) > 0:
        pd.DataFrame(fails).to_csv(f'../data/errors/errors_batch_final.csv')
    
    print('                                                                                              ', end='\r')
    print('Collection Complete!')

In [8]:
pull_data(steam_games['appid'], batch_size=1000)

Collection Complete!                                                                          


In [9]:
def consolidate_batches():
    files = os.listdir('../data/batches/')
    df = pd.read_csv('../data/batches/batch0.csv')
    
    for i in range(1, len(files)-1):
        new_batch = pd.read_csv(f'../data/batches/batch{i}.csv')
        df = pd.concat([df,new_batch], axis=0)
        
    final_batch = pd.read_csv('../data/batches/batch_final.csv')
    df = pd.concat([df,final_batch], axis=0)
    
    df.to_csv('../data/steam_games.csv', index=False)

In [10]:
consolidate_batches()

In [11]:
store_json_showcase = {
    'type': 'game',
    'name': 'Counter-Strike',
    'steam_appid': 10,
    'required_age': 0,
    'is_free': False,
    'detailed_description': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.",
    'about_the_game': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.",
    'short_description': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.",
    'supported_languages': 'English<strong>*</strong>, French<strong>*</strong>, German<strong>*</strong>, Italian<strong>*</strong>, Spanish - Spain<strong>*</strong>, Simplified Chinese<strong>*</strong>, Traditional Chinese<strong>*</strong>, Korean<strong>*</strong><br><strong>*</strong>languages with full audio support',
    'header_image': 'https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1602535893',
    'website': None,
    'pc_requirements': {'minimum': '\r\n\t\t\t<p><strong>Minimum:</strong> 500 mhz processor, 96mb ram, 16mb video card, Windows XP, Mouse, Keyboard, Internet Connection<br /></p>\r\n\t\t\t<p><strong>Recommended:</strong> 800 mhz processor, 128mb ram, 32mb+ video card, Windows XP, Mouse, Keyboard, Internet Connection<br /></p>\r\n\t\t\t'},
    'mac_requirements': {'minimum': 'Minimum: OS X  Snow Leopard 10.6.3, 1GB RAM, 4GB Hard Drive Space,NVIDIA GeForce 8 or higher, ATI X1600 or higher, or Intel HD 3000 or higher Mouse, Keyboard, Internet Connection'},
    'linux_requirements': {'minimum': 'Minimum: Linux Ubuntu 12.04, Dual-core from Intel or AMD at 2.8 GHz, 1GB Memory, nVidia GeForce 8600/9600GT, ATI/AMD Radeaon HD2600/3600 (Graphic Drivers: nVidia 310, AMD 12.11), OpenGL 2.1, 4GB Hard Drive Space, OpenAL Compatible Sound Card'},
    'developers': ['Valve'],
    'publishers': ['Valve'],
    'price_overview': {
        'currency': 'USD',
        'initial': 999,
        'final': 999,
        'discount_percent': 0,
        'initial_formatted': '',
        'final_formatted': '$9.99'
    },
    'packages': [574941, 7],
    'package_groups': [
        {
            'name': 'default',
            'title': 'Buy Counter-Strike',
            'description': '',
            'selection_text': 'Select a purchase option',
            'save_text': '',
            'display_type': 0,
            'is_recurring_subscription': 'false',
            'subs': [
                {
                    'packageid': 7,
                    'percent_savings_text': ' ',
                    'percent_savings': 0,
                    'option_text': 'Counter-Strike: Condition Zero - $9.99',
                    'option_description': '',
                    'can_get_free_license': '0',
                    'is_free_license': False,
                    'price_in_cents_with_discount': 999
                },
                         
                {
                    'packageid': 574941,
                    'percent_savings_text': ' ',
                    'percent_savings': 0,
                    'option_text': 'Counter-Strike - Commercial License - $9.99',
                    'option_description': '',
                    'can_get_free_license': '0',
                    'is_free_license': False,
                    'price_in_cents_with_discount': 999
                }
            ]
        }
    ],
    'platforms': {
        'windows': True,
        'mac': True,
        'linux': True
    },
    'metacritic': {
        'score': 88,
        'url': 'https://www.metacritic.com/game/pc/counter-strike?ftag=MCD-06-10aaa1f'
    },
    'categories': [
        {
            'id': 1,
            'description': 'Multi-player'
        },
        {
            'id': 49,
            'description': 'PvP'
        },
        {
            'id': 36, 
            'description': 'Online PvP'
        },
        {
            'id': 37,
            'description': 'Shared/Split Screen PvP'
        },
        {
            'id': 8,
            'description': 'Valve Anti-Cheat enabled'
        }
    ],
    'genres': [
        {
            'id': '1',
            'description': 'Action'
        }
    ],
    'screenshots': [
        {
            'id': 0,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000000132.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000000132.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 1,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000000133.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000000133.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 2,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000000134.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000000134.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 3,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000000135.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000000135.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 4,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000000136.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000000136.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 5,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002540.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002540.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 6,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002539.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002539.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 7,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002538.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002538.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 8,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002537.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002537.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 9,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002536.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002536.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 10,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002541.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002541.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 11,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002542.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002542.1920x1080.jpg?t=1602535893'
        },
        {
            'id': 12,
            'path_thumbnail': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002543.600x338.jpg?t=1602535893',
            'path_full': 'https://cdn.akamai.steamstatic.com/steam/apps/10/0000002543.1920x1080.jpg?t=1602535893'
        }
    ],
    'recommendations': {
        'total': 111466
    },
    'release_date': {
        'coming_soon': False, 
        'date': 'Nov 1, 2000'
    },
    'support_info': {
        'url': 'http://steamcommunity.com/app/10',
        'email': ''
    },
    'background': 'https://cdn.akamai.steamstatic.com/steam/apps/10/page_bg_generated_v6b.jpg?t=1602535893',
    'content_descriptors': {
        'ids': [2, 5],
        'notes': 'Includes intense violence and blood.'
    }
}

In [12]:
spy_json_showcase = {
    'appid': 10,
    'name': 'Counter-Strike',
    'developer': 'Valve',
    'publisher': 'Valve',
    'score_rank': '',
    'positive': 184272,
    'negative': 4785,
    'userscore': 0,
    'owners': '10,000,000 .. 20,000,000',
    'average_forever': 11526,
    'average_2weeks': 720,
    'median_forever': 207,
    'median_2weeks': 1085,
    'price': '999',
    'initialprice': '999',
    'discount': '0',
    'ccu': 14884,
    'languages': 'English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean',
    'genre': 'Action',
    'tags': {
        'Action': 5371,
        'FPS': 4795,
        'Multiplayer': 3356,
        'Shooter': 3321,
        'Classic': 2751,
        'Team-Based': 1839,
        'First-Person': 1688,
        'Competitive': 1585,
        'Tactical': 1318,
        "1990's": 1177,
        'e-sports': 1170,
        'PvP': 861,
        'Old School': 744,
        'Military': 622,
        'Strategy': 607,
        'Survival': 294,
        'Score Attack': 285,
        '1980s': 256,
        'Assassin': 222,
        'Violent': 63
    }
}